# Setup

Import necessary modules and do some basic setup.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta

# Custom utils for processing the data
from utils_data import * 
from utils_ml import *


# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Custom utils
from utils_data import *

In [ ]:
# Define some functions for plotting
def plotprediction_TS(test_dates, final_predictions, test_labels):
    import seaborn as sns
    df_to_compare = pd.DataFrame({'date': test_dates, 'Actual': test_labels, 'Predicted': final_predictions})
    dfm = pd.melt(df_to_compare, id_vars=['date'], value_vars=['Actual', 'Predicted'], var_name='data', value_name='precip')
    f, axs = plt.subplots(1,2,
                      figsize=(12,5),
                      sharey=True)

    sns.regplot(data= df_to_compare,
                x="Actual",
                y="Predicted",
                ax=axs[0],
                )
    sns.lineplot(x='date', y='precip', hue = 'data', data=dfm, ax=axs[1])

In [ ]:
def plot_result(x, y):

    fig, ax = plt.subplots()
    ax.scatter(test_labels, preds)
    ax.plot([test_labels.min(), test_labels.max()], [test_labels.min(), test_labels.max()], 'k--', lw=1)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()

Define some paths and constants.

In [ ]:
# Paths
DATADIR = os.getcwd() + '/../data'

# Some constants
DATE_START = '1979-01-01'
DATE_END = '2020-12-31'


# Unsupervised learning approaches

## Getting started with the data

In [ ]:
#Open data
mslp = xr.open_mfdataset(DATADIR + '/ERA5/Day_era5_2deg_MSL_EU_1979-2021.nc', combine='by_coords')

mslp = mslp.sel(time=slice(DATE_START, DATE_END))

# Convert to hPa
mslp.MSL.values = mslp.MSL.values/100
lon = mslp.lon
lat = mslp.lat
dates = mslp.time


In [ ]:
# Plot an example (day) of mslp
mslp.MSL.isel(time=200).plot();

In [ ]:
# Process season mean
seas_means = mslp.groupby("time.season").mean()

fg = seas_means.MSL.plot(col="season",  col_wrap=4,
    # The remaining kwargs customize the plot just as for not-faceted plots
    robust=True,
    cmap=mpl.cm.RdYlBu_r)

# Use this to plot contours on each panel
# Note that this plotting call uses the original DataArray gradients
fg.map_dataarray(
    xr.plot.contour, x="lon", y="lat", colors="k", levels=13, add_colorbar=False
)

In [ ]:
# Compute anomalies
climatology = mslp.mean('time')

# By season
season_climatology = mslp.groupby('time.season').mean('time')

# Climatological anomalies
anom_mslp =  mslp.MSL  - climatology

# By season
anom_seas_mslp = mslp.groupby('time.season') - season_climatology

## PCA

In [ ]:
# Start using the whole data set for PCA. Then, anomalies can be used
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# We need to reshape the data [time,latxlon]  
mslp_stacked = mslp.stack(latlon=('lat', 'lon'))

# Load in memory for computing the PCA
mslp_stacked.load()

# Extract msl variable
X = mslp_stacked.MSL

In [ ]:
# The number of variables (features) is 1025 (41 points in longitude * 25 points in latitude)
# Standardise the data
from sklearn.preprocessing import StandardScaler

scaler  = StandardScaler()
scaler = scaler.fit(X)
X = scaler.transform(X)

In [ ]:
# Do the PCA
pca = PCA()
pca.fit(X)

cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [ ]:
# See how many components 
f, ax = plt.subplots(figsize=(6,6))
ax.plot(range(1,21), pca.explained_variance_ratio_[0:20]*100)
ax.plot(range(1,21), pca.explained_variance_ratio_[0:20]*100,'ro')
ax.grid(ls=':')
ax.set_xticks(range(1,21)); 
ax.set_xlabel('PC#');
ax.set_ylabel("% variance");

In [ ]:
# We can take 4 or 5
# Following the literature I will take 4 (e.g. Cortesi et a., 2021)
n = 12 # We can change this number
pca.explained_variance_ratio_[:n].sum()

In [ ]:
PCs = pca.fit_transform(X)
PCs_n = PCs[:,:n]

# Data frame format for the selected components
PCdf = pd.DataFrame(PCs_n, index = mslp['time'], \
                    columns = ["PC%s" % (x) for x in range(1, PCs_n.shape[1] +1)])

# See the data
PCdf.head()

The EOFS (Empirical orthogonal functions) contain the spatial patterns associated with each PC

In [ ]:
EOFs = pca.components_
EOFs = EOFs[:n,:]

# Reshape the data
EOFs_r = EOFs.reshape((n, len(lat), len(lon)))
EOFs_r.shape

In [ ]:
nn = []
tot_var = []
for ip in range(n):
    xn = pca.explained_variance_ratio_[:ip + 1].sum()
    nn.append(xn)
    xx =  pca.explained_variance_ratio_[:ip + 1].sum() - pca.explained_variance_ratio_[:ip ].sum()
    tot_var.append(xx)

In [ ]:
# Convert into Xarray for visualization
XD_EOFs_r = xr.DataArray(data=EOFs_r, coords=[("PCA", tot_var), ("lat", lat.data), ("lon", lon.data)])

fg = XD_EOFs_r.plot(col="PCA",  col_wrap=4,
    # The remaining kwargs customize the plot just as for not-faceted plots
    robust=True,
    cmap=mpl.cm.RdYlBu_r)

# Use this to plot contours on each panel
# Note that this plotting call uses the original DataArray gradients
fg.map_dataarray(
    xr.plot.contour, x="lon", y="lat", colors="k", levels=13, add_colorbar=False
)

## K-means clustering

In [ ]:
from sklearn.cluster import KMeans

# Perform K-cluster analysis using the PCds obtained before
nclusters = 12
kmeans = KMeans(init='k-means++', n_clusters=nclusters, n_init=10)
kmeans.fit(PCdf.values)
y_pred = kmeans.fit_predict(PCdf.values)

# Each day belongs to a cluster, labelled by kmeands.labels_
np.unique(kmeans.labels_)

In [ ]:
labels = pd.DataFrame(kmeans.labels_, index=np.array(mslp['time']), columns=['cluster'])

In [ ]:
#labels = pd.DataFrame(kmeans.labels_, index=mslp['time'], columns=['cluster'])

# See how many days belong to cluster 0
index = labels.query('cluster == {}'.format(0))
len(index)

For each cluster we calculate the mean 

In [ ]:
num_tot = len(labels.cluster)
clusters = []
day_clusters = []
nbdays = []

for iclus in range(nclusters): 
    index = labels.query('cluster == {}'.format(iclus)) 
    freq = (len(index)/num_tot)*100
    freq = round(freq,2)
    nbdays.append(freq)
    cluster = mslp.sel(time=index.index).mean('time')
    Dcluster = mslp.sel(time=index.index)
    clusters.append(cluster)
    day_clusters.append(Dcluster)
    
clusters = xr.concat(clusters, dim='cluster')
clusters.assign_coords(cluster=nbdays)

#day_clusters = xr.concat(day_clusters, dim='cluster')
#day_clusters.assign_coords(day_clusters=nbdays)


In [ ]:
fg_C = clusters.MSL.plot(col="cluster",  col_wrap=4,
    # The remaining kwargs customize the plot just as for not-faceted plots
    robust=True, 
    cmap=mpl.cm.RdYlBu_r)

fg_C.map_dataarray(
    xr.plot.contour, x="lon", y="lat", colors="k", levels=13, add_colorbar=False
)

In [ ]:
# Save PCs and Clusters 
PCdf['date'] = PCdf.index
PCdf.to_csv(DATADIR + 'PCdf.csv',index=False)

In [ ]:
def prepare_ClusterData(day_clusters):
    t_list = []
    for iclus in range(0,len(day_clusters)):
        print(iclus)
        tmp = day_clusters[iclus].mean(dim=['lon', 'lat'])
        tmp_df = pd.DataFrame({'date' : tmp['time'], 'MSL': tmp['MSL'], 'Cluster': iclus})
        t_list.append(tmp_df)

    # Merge by date 
    df = pd.concat(t_list)
    df = df.sort_values(by="date")
    return(df)


    

In [ ]:
Cluster_spatialmean = prepare_ClusterData(day_clusters)
Cluster_spatialmean

In [ ]:
Cluster_spatialmean.to_csv(DATADIR + 'Cluster_spatialmean.csv', index=False)

# Supervised learning approaches

## Data preparation: precipitation time series

**Dataset**: RhiresD, which is a gridded daily precipitation dataset over Switzerland provided by MeteoSwiss. It is based on a spatial interpolation of rain-gauge data. The grid resolution is 1 km, but the effective resolution is in the order of 15-20 km.


**Aggregations levels**: The gridded dataset has been averaged over different regions:
* 12 climatic regions
* 5 aggregated regions
* the whole country

In [ ]:
precip = get_precipitation_data(DATADIR + '/MeteoSwiss/precip_regions.csv', DATE_START, DATE_END)

precip_p95 = precip_exceedance(precip, 0.95)
precip_p99 = precip_exceedance(precip, 0.99)

In [ ]:
variables = read_csv_files(glob.glob(os.path.join(
    DATADIR + '/ERA5/TS_CH/', '*.csv')), DATE_START, DATE_END)

full_data = concat_dataframes([variables, precip.reg_tot])

full_data

#### Data used: 
A set of primary meteorological variables are used as covariates.
- Geopotential levels (1000,850,700,500,300)
- MSL: Mean sea level pressure
- T2MMEAN: 2m-temperature

**Extra-variables such as lagged T2MMEAN and PCs will be used (see below)**

In [ ]:
# Prepare the dataset 
DIRCSV  = DATADIR + '/TS_CH/'
l_files = glob.glob(os.path.join(DIRCSV, 'df*.csv'))
df_vars =  read_csv_files(l_files, DATE_START, DATE_END)
df_vars.head()

In [ ]:
# Add lag-t2mmean: For precipiation the memory system is important, adding lagged variables might help to predict precipitation
df_vars['T2MLag'] = df_vars['T2MMEAN'].shift(1)
df_vars.head()

DATADIR### Use PCs from the previous steps (PCA-analysis)

In [ ]:
DATADIR

In [ ]:
# Read Large scale-atmospheric PCs and Clusters
df_PCs = pd.read_csv(DATADIR + '/ERA5/PCdf.csv')
df_PCs['date'] = pd.DatetimeIndex(df_PCs['date']).normalize()
df_clusters =  pd.read_csv(DATADIR + '/ERA5/Cluster_spatialmean.csv')
df_clus    = df_clusters[['date','Cluster']]
df_clus['date'] = pd.DatetimeIndex(df_clus['date']).normalize()

In [ ]:
# TARGET: read precipitaion
df_prec = get_precipitation_data ( DATADIR + '/TS_CH/precip_regions.csv', DATE_START, DATE_END)
# Select the right columns: date and reg_tot (all country)
df_prec = df_prec[['date','reg_tot']]

In [ ]:
# Merge everything
l_all = []
l_all.append(df_vars)
l_all.append(df_prec)
df_all = concat_dataframes(l_all)

In [ ]:
#Data exploration
import seaborn
seaborn.pairplot(df_all, vars=df_all.columns[1:9],
                 kind='reg')  

In [ ]:
yy_train = [1979,2015]
yy_test  = [2016,2020]
ylabel = df_prec.columns[1]

# Add categorical variables
#df_input = pd.merge(df_all, df_clus)
df_input = pd.merge(df_all, df_PCs)
names_col = df_input.columns
# define attributes - i.e covariates: I will also remove MSL as I am using the PCs
attributes = names_col.drop(['date','reg_tot','MSL'])
df_input.head()

In [ ]:
# Visualize the target variable: precipitation
# Plot Time series
from matplotlib import pyplot
df_input['reg_tot'].plot()
pyplot.show()

In [ ]:
train_dataset, train_labels, test_dataset, test_labels, train_dates, test_dates = split_data(df_input, yy_train, yy_test, attributes, ylabel)

In [ ]:
# cat_var='Cluster' # in case of using the clusters, it has to be a categorical variable.
# In the following we won't use categorical variables, as we're using the PCs
fpipeline = prepareData(train_dataset, None)
X_prep_train = fpipeline.fit_transform(train_dataset)
X_prep_test = fpipeline.fit_transform(test_dataset)

In [ ]:
X_prep_train

## 1. Multiple Linear regression for precipitation values

In [ ]:
# Import some libraries
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

In [ ]:
lr = LinearRegression(n_jobs=16)
lr.fit(X_prep_train, train_labels)


In [ ]:
mse_train = mean_squared_error(train_labels, lr.predict(X_prep_train))
mse_test = mean_squared_error(test_labels, lr.predict(X_prep_test))
print(f'Train MSE = {mse_train}'); print(f'Test MSE = {mse_test}')
print(f'Train RMSE = {np.sqrt(mse_train)}'); print(f'Test RMSE = {np.sqrt(mse_test)}')

In [ ]:
rfe = RFE(lr)             
rfe = rfe.fit(X_prep_train, train_labels)
mean_squared_error(train_labels, rfe.predict(X_prep_train))

In [ ]:
# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validated:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
lr_cv_mse = cross_val_score(lr, X_prep_train, train_labels, scoring='neg_mean_squared_error', cv=10)
# We got the negative average MSE for cross-validation (minimizing MSE is equivalent to maximizing the negative MSE)
lr_cv_mse.mean()
# The result is close to what we obtained before. The negative result 

In [ ]:
lin_rmse_scores = np.sqrt(-lr_cv_mse)
pd.Series(lin_rmse_scores).describe()

In [ ]:
print("Folds: " + str(len(lr_cv_mse)) + ", MSE: " + str(np.mean(np.abs(lr_cv_mse))) + ", STD: " + str(np.std(lr_cv_mse)))


In [ ]:
# get the coefficients
lr.coef_
#coeff_df = pd.DataFrame(lr.coef_, attributes, columns=['Coefficient'])
# makes some predictions
y_pred = lr.predict(X_prep_test)

In [ ]:
plotprediction_TS(test_dates, y_pred, test_labels)

In [ ]:
# Since we're getting negative predicted values, let's try to transform the target variable 
# Transform targets and use same linear model
from sklearn.compose import TransformedTargetRegressor
regr_trans = TransformedTargetRegressor(regressor=LinearRegression(),
                                        func=np.log1p,
                                        inverse_func=np.expm1)


regr_trans.fit(X_prep_train, train_labels)
y_trans_pred = regr_trans.predict(X_prep_test)

In [ ]:
mse_transf_train = mean_squared_error(train_labels, regr_trans.predict(X_prep_train))
mse_transf_test = mean_squared_error(test_labels, regr_trans.predict(X_prep_test))
print(f'Train MSE = {mse_transf_train}'); print(f'Test MSE = {mse_transf_test}')
print(f'Train RMSE = {np.sqrt(mse_transf_train)}'); print(f'Test RMSE = {np.sqrt(mse_transf_test)}')

In [ ]:
#Plot predictions
plotprediction_TS(test_dates, y_trans_pred, test_labels)

*The predictions seem to be better, but the MSE are still large*

## 2. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(X_prep_train, train_labels)
# make predictions
y_rf_pred = forest_reg.predict(X_prep_test)

In [ ]:
mse_rf_train = mean_squared_error(train_labels, forest_reg.predict(X_prep_train))
mse_rf_test = mean_squared_error(test_labels, forest_reg.predict(X_prep_test))
print(f'Train MSE = {mse_rf_train}'); print(f'Test MSE = {mse_rf_test}')
print(f'Train RMSE = {np.sqrt(mse_rf_train)}'); print(f'Test RMSE = {np.sqrt(mse_rf_test)}')

In [ ]:
# We can see overfitting problem

In [ ]:
# Tunning parameter
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]
# Create the parameter grid based on the results of random search 
    
forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_prep_train, train_labels)
best_params = grid_search.best_params_

In [ ]:
print ("Best Score:" , grid_search.best_score_)
print ("Best params:", best_params)

In [ ]:
forest_GCV_reg = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
forest_GCV_reg.fit(X_prep_train,train_labels)

In [ ]:
y_rf_cv_predict = forest_GCV_reg.predict(X_prep_test)
mse_rf_cv_train = mean_squared_error(train_labels, forest_GCV_reg.predict(X_prep_train))
mse_rf_cv_test = mean_squared_error(test_labels, forest_GCV_reg.predict(X_prep_test))
print(f'Train MSE = {mse_rf_cv_train}'); print(f'Test MSE = {mse_rf_cv_test}')
print(f'Train RMSE = {np.sqrt(mse_rf_cv_train)}'); print(f'Test RMSE = {np.sqrt(mse_rf_cv_test)}')

In [ ]:
plotprediction_TS(test_dates, y_rf_cv_predict, test_labels)

In [ ]:
features_importance = forest_GCV_reg.feature_importances_
sorted_features_importance = sorted(zip(features_importance, attributes), reverse=True)
sorted_features_importance

In [ ]:
indices = np.argsort(features_importance)
plt.barh(range(len(attributes)), features_importance[indices], color='b', align='center')
plt.yticks(range(len(indices)), [attributes[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

# Predicting extremes

## 1. Logistic Regression

In [ ]:
# define exceedances based on the 95th
df_prec_ex = precip_exceedance(df_prec)
df_prec_ex['reg_tot'] = df_prec_ex['reg_tot']*1

In [ ]:
df_input_ex = df_input
# Replace reg_tot by the exceedances
df_input_ex['reg_tot'] = df_prec_ex['reg_tot']

In [ ]:
train_dataset, train_labels, test_dataset, test_labels, train_dates, test_dates = split_data(df_input_ex, yy_train, yy_test, attributes, ylabel)
# but the data is already in the format (only the labels have been replace by the exceedances)

In [ ]:
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_prep_train, train_labels)

In [ ]:
y_ex_pred=logisticRegr.predict(X_prep_test)

In [ ]:
# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(test_labels, y_ex_pred)
print(cnf_matrix)

In [ ]:
# there are 7 false positive, and 70  (70+7)

In [ ]:
# Use score method to get accuracy of model
score = logisticRegr.score(X_prep_test, test_labels)
print(score)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, y_ex_pred, target_names=['0', '1']))

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve

# Make probability predictions
train_probs = logisticRegr.predict_proba(X_prep_train)[:, 1]
probs = logisticRegr.predict_proba(X_prep_test)[:, 1]

train_predictions = logisticRegr.predict(X_prep_train)
predictions = logisticRegr.predict(X_prep_test)
print(f'Train ROC AUC Score: {roc_auc_score(train_labels, train_probs)}')
print(f'Test ROC AUC  Score: {roc_auc_score(test_labels, probs)}')

In [ ]:
evaluate_model(test_labels,train_labels, predictions, probs, train_predictions, train_probs)

## Method 2. Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
rf_class = RandomForestClassifier(n_estimators=150, 
                               random_state=42, 
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 1)

# Fit on training data
rf_class.fit(X_prep_train, train_labels)

In [ ]:
n_nodes = []
max_depths = []

for ind_tree in rf_class.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
# evaluate the results

In [ ]:
train_rf_predictions = rf_class.predict(X_prep_train)
train_rf_probs = rf_class.predict_proba(X_prep_train)[:, 1]


In [ ]:
rf_predictions = rf_class.predict(X_prep_test)
rf_probs = rf_class.predict_proba(X_prep_test)[:, 1]

In [ ]:
evaluate_model(test_labels, train_labels, rf_predictions, rf_probs,train_rf_predictions, train_rf_probs )